In [1]:
import pickle

import pandas as pd
import numpy as np

# Import features

In [2]:
#Import training features
import pathlib
path_to_read_model = 'C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\my_env_codebasics\\My_CODE\\ML-projects\\advanced_House_Price_Prediction-Regression\\probleam_study' #Path of current working Directory
with open(path_to_read_model + '\\trainFeatures_list.pkl', 'rb') as f:
    X, y = pickle.load(f)

# PreProcessing Objects needed for Transformations

In [3]:
location_stats_greater_than_10 = (X['location'].apply(lambda x: str(x).strip()).value_counts(ascending=False, dropna=False)[X['location'].apply(lambda x: str(x).strip()).value_counts(ascending=False, dropna=False)>10]).index.unique()
location_stats_greater_than_10

Index(['Whitefield', 'Sarjapur  Road', 'Electronic City', 'Kanakpura Road',
       'Thanisandra', 'Hebbal', 'Marathahalli', 'Uttarahalli', 'Yelahanka',
       'Raja Rajeshwari Nagar',
       ...
       'Kalyan nagar', 'Battarahalli', 'Kothannur', 'Kambipura',
       'Kumaraswami Layout', 'Anjanapura', 'Banaswadi', 'Hosakerehalli',
       'HRBR Layout', 'BTM 2nd Stage'],
      dtype='object', length=117)

# Saving the PreProcessing Objects Using Pickel

In [4]:
preProcessingObjects_list = [location_stats_greater_than_10]

import pathlib
path_to_write_output=str(pathlib.Path.cwd()) #Path of current working Directory
with open(path_to_write_output + '\\preProcessingObjects_list.pkl', 'wb') as handle:
    pickle.dump(preProcessingObjects_list, handle)

# Data Transformation

In [5]:
def Transformations(featureDF, targetDF):
    # Load the PreProcessing Objects needed for Transformations
    #Import training features
    import pathlib
    path_to_read_model = 'C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\my_env_codebasics\\My_CODE\\ML-projects\\advanced_House_Price_Prediction-Regression\\preProcessingObjects_list' #Path of current working Directory
    with open(path_to_read_model + '\\preProcessingObjects_list.pkl', 'rb') as f:
        preProcessingObjects_list = pickle.load(f)
    location_stats_greater_than_10 = preProcessingObjects_list[0]
    
    
    # Feature Selection via Business knoledge. Society is dependednt on location
    featureDF.drop(['society'], axis=1, inplace=True)
    
    # Duplicate elimination
    featureDF.drop_duplicates(inplace=True)
    # Update y matrix based X
    ## since we've removed some data from X, we need to pass on these updations to y as well, as y doesn't know some of its corresponding X's have been deleted.
    targetDF = targetDF[featureDF.index]
    
    # convert cat-col of total_sqft to float with null values also
    def convert_sqft_to_num(curr_tuple):
        try:
            tokens = curr_tuple.split('-')
            if len(tokens) == 2:
                return (float(tokens[0])+float(tokens[1]))/2
            return float(curr_tuple)
        except:
            return np.NaN
    
    featureDF['total_sqft'] = featureDF.total_sqft.apply(convert_sqft_to_num)
    
    # convert cat-col of availability to make both Immediate Possession and Ready To Move same and take only month, include null also.
    def convert_availability(curr_tuple):
        try:
            curr_tuple = curr_tuple.lower()
            if curr_tuple == 'ready to move' or curr_tuple == 'immediate possession':
                return 'available_currently'
            tokens = curr_tuple.split('-')
            if len(tokens) == 2:
                return tokens[1].strip()
        except:
            return np.NaN
        
    featureDF['availability'] = featureDF.availability.apply(convert_availability)
    
    # convert cat-col of size to number by the frist value with null value also
    def convert_size_to_num(curr_tuple):
        try:
            tokens = curr_tuple.split(' ')
            return float(tokens[0])
        except:
            return np.NaN
        
    featureDF['size'] = featureDF['size'].apply(convert_size_to_num)
    
    # Convert location to reduce the unique values in the column
    X['location'] = X['location'].apply(lambda x: x if (x in location_stats_greater_than_10) else 'other')
    
    # Update y matrix based X
    ## since we've removed some data from X, we need to pass on these updations to y as well, as y doesn't know some of its corresponding X's have been deleted.
    targetDF = targetDF[featureDF.index]
    
    return featureDF, targetDF

# Saving the Transformation function Using Pickel

In [6]:
import pathlib
path_to_write_output=str(pathlib.Path.cwd()) #Path of current working Directory
with open(path_to_write_output + '\\transformation_function.pkl', 'wb') as handle:
    pickle.dump(Transformations, handle)

# Transforming features and Target

In [7]:
# Load the PreProcessing Objects needed for Transformations
#Import training features
import pathlib
path_to_read_model = 'C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\my_env_codebasics\\My_CODE\\ML-projects\\advanced_House_Price_Prediction-Regression\\preProcessingObjects_list' #Path of current working Directory
with open(path_to_read_model + '\\transformation_function.pkl', 'rb') as f:
    transformation_function = pickle.load(f)

X, y = transformation_function(X, y)

In [8]:
#Check for null values in the Train and Test Data
print("Null values in the Training feature: ")
display(X.isnull().sum())
#print("Null values in the Test feature: ")
#display(X_test.isnull().sum())

print("-"*124)

#Check for null values in the Target
print("Null values in the Training feature: ")
display(y.isnull().sum())
#print("Null values in the Test feature: ")
#display(y_test.isnull().sum())

print("-"*124)

#Check for DataSets Shape
print("Shape of Training Features: ",X.shape)
print("Shape of Training target: ",y.shape)
#print("Shape of Testing Features: ",X_test.shape)
#print("Shape of Testing target: ",y_test.shape)

Null values in the Training feature: 


area_type         0
availability      0
location          0
size              4
total_sqft       15
bath             25
balcony         264
dtype: int64

----------------------------------------------------------------------------------------------------------------------------
Null values in the Training feature: 


0

----------------------------------------------------------------------------------------------------------------------------
Shape of Training Features:  (4928, 7)
Shape of Training target:  (4928,)


# Outlier Specifications based on IQR

In [9]:
num_cols = [col for col in X.columns if X[col].dtypes!='O']
print(">>Numerical Columns in the Data: \n",num_cols)
cat_cols = [col for col in X if X[col].dtypes == 'O']
print(">>Catogorical Columns are: \n",cat_cols)

>>Numerical Columns in the Data: 
 ['size', 'total_sqft', 'bath', 'balcony']
>>Catogorical Columns are: 
 ['area_type', 'availability', 'location']


In [10]:
print(">>numerical columns in analysis: ", num_cols)
outlier_Dict = dict()
for col in num_cols:
    X_min = X[col].min()
    X_max = X[col].max()
    median = X[col].median()
    q1 = X[col].quantile(0.25)
    q3 = X[col].quantile(0.75)
    iqr = q3 - q1
    upper_limit = q3 + 1.5 * iqr
    lower_limit = q1 - 1.5 * iqr
    outlier_Dict[col] = {'25quantile':q1, '75quantile':q3, 
                         'iqr':iqr, 'min': X_min, 'max':X_max, 'median':median, 
                         'lower_limit':lower_limit, 'upper_limit':upper_limit}
    
#Check the Outlier Specifics
import json
print(">>Outlier IQR results:\n",json.dumps(outlier_Dict,sort_keys=False, indent=4, default=str))    

>>numerical columns in analysis:  ['size', 'total_sqft', 'bath', 'balcony']
>>Outlier IQR results:
 {
    "size": {
        "25quantile": 2.0,
        "75quantile": 3.0,
        "iqr": 1.0,
        "min": 1.0,
        "max": 27.0,
        "median": 3.0,
        "lower_limit": 0.5,
        "upper_limit": 4.5
    },
    "total_sqft": {
        "25quantile": 1100.0,
        "75quantile": 1690.0,
        "iqr": 590.0,
        "min": 1.0,
        "max": 52272.0,
        "median": 1296.0,
        "lower_limit": 215.0,
        "upper_limit": 2575.0
    },
    "bath": {
        "25quantile": 2.0,
        "75quantile": 3.0,
        "iqr": 1.0,
        "min": 1.0,
        "max": 27.0,
        "median": 2.0,
        "lower_limit": 0.5,
        "upper_limit": 4.5
    },
    "balcony": {
        "25quantile": 1.0,
        "75quantile": 2.0,
        "iqr": 1.0,
        "min": 0.0,
        "max": 3.0,
        "median": 2.0,
        "lower_limit": -0.5,
        "upper_limit": 3.5
    }
}


# Save the Outlier Specification to JSON

In [11]:
with open('outlier_Dict.json', 'w') as fp:
    json.dump(outlier_Dict, fp)

# Outlier Removal functions

In [12]:
def outlierRemoval(featureDF, targetDF):
    outlier_Dict_DirPath = 'C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\my_env_codebasics\\My_CODE\\ML-projects\\advanced_House_Price_Prediction-Regression\\preProcessingObjects_list'
    with open(outlier_Dict_DirPath+'\\outlier_Dict.json') as json_file:
        outlier_Dict = json.load(json_file)
    
    for column in outlier_Dict:
        lower_limit =outlier_Dict[column]['lower_limit']
        upper_limit =outlier_Dict[column]['upper_limit']
        print("column: %s, lower_limit: %s, upper_limit: %s"%(column, lower_limit, upper_limit))
        featureDF = featureDF[((featureDF[column]>lower_limit)&(featureDF[column]<upper_limit))| (featureDF[column].isna())]
    
    # Update y matrix based X
    ## since we've removed some data from X, we need to pass on these updations to y as well, as y doesn't know some of its corresponding X's have been deleted.
    targetDF = targetDF[featureDF.index]
    
    return featureDF, targetDF    

# Saving the OutlierRemoval function Using Pickel

In [13]:
import pathlib
path_to_write_output=str(pathlib.Path.cwd()) #Path of current working Directory
with open(path_to_write_output + '\\outlierRemoval_function.pkl', 'wb') as handle:
    pickle.dump(outlierRemoval, handle)

# Handeling Outlier of features and Target

In [14]:
# Load the PreProcessing Objects needed for Transformations
#Import training features
import pathlib
path_to_read_model = 'C:\\Users\\koriv\\Desktop\\MachineLearning_DataScience\\Hands_On_Machine_Learning\\my_env_codebasics\\My_CODE\\ML-projects\\advanced_House_Price_Prediction-Regression\\preProcessingObjects_list' #Path of current working Directory
with open(path_to_read_model + '\\outlierRemoval_function.pkl', 'rb') as f:
    outlierRemoval_function = pickle.load(f)

X, y = outlierRemoval_function(X, y)

column: size, lower_limit: 0.5, upper_limit: 4.5
column: total_sqft, lower_limit: 215.0, upper_limit: 2575.0
column: bath, lower_limit: 0.5, upper_limit: 4.5
column: balcony, lower_limit: -0.5, upper_limit: 3.5


In [15]:
#Check for null values in the Train and Test Data
print("Null values in the Training feature: ")
display(X.isnull().sum())


print("-"*124)

#Check for null values in the Target
print("Null values in the Training feature: ")
display(y.isnull().sum())


print("-"*124)

#Check for DataSets Shape
print("Shape of Training Features: ",X.shape)
print("Shape of Training target: ",y.shape)

Null values in the Training feature: 


area_type         0
availability      0
location          0
size              3
total_sqft       12
bath             16
balcony         124
dtype: int64

----------------------------------------------------------------------------------------------------------------------------
Null values in the Training feature: 


0

----------------------------------------------------------------------------------------------------------------------------
Shape of Training Features:  (4225, 7)
Shape of Training target:  (4225,)


In [16]:
X.describe()

,size,total_sqft,bath,balcony
count,4222.000000,4213.000000,4209.000000,4101.000000
mean,2.485315,1324.157277,2.337372,1.567910
std,0.704100,409.554657,0.702882,0.792393
min,1.000000,250.000000,1.000000,0.000000
25%,2.000000,1088.000000,2.000000,1.000000
50%,2.000000,1250.000000,2.000000,2.000000
75%,3.000000,1548.300000,3.000000,2.000000
max,4.000000,2572.000000,4.000000,3.000000
